In [49]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results
import csv
import pandas
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob 
%matplotlib inline
import re
import time
import math

In [50]:
#setting up credentials

In [51]:
enterprise_search_args = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_api",
                                          env_overwrite=False)

Grabbing bearer token from OAUTH


In [ ]:
#enter the data

In [57]:
data = pandas.read_csv("~/Documents/nfl sentiment/final_subset.csv")

In [58]:
#looking at data head

In [59]:
data.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
0,Russell Wilson,@DangeRussWilson',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Russell Wilson OR @DangeRussWilson'
1,Doug Baldwin,@DougBaldwinJr',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Doug Baldwin OR @DougBaldwinJr'
2,Jimmy Graham,@TheJimmyGraham',Seattle Seahawks,W,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Jimmy Graham OR @TheJimmyGraham'
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'


In [ ]:
#creating function to filter for teams

In [108]:
def get_data(teams):
    data_1 = data[data.Team.isin(teams)]
    
    return data_1

In [ ]:
#creating function to get tweets

In [110]:
def get_tweets(data):
    
    #getting the twitter handles
    twitter = data.full_twitter.tolist()

    newtwitter = []

    for i in range(len(twitter)):
        a = twitter[i].replace("'", "")
    
        newtwitter.append(a)
    
    print(newtwitter)
    
    
    
    #getting start date
    
    start = data.T1_start.tolist()

    start = start[1].replace("'", "")
    
    print(start)
    
    #getting end date
    
    end = data.T1_end.tolist()

    end = end[1].replace("'", "")
    
    print(end)
    
    
    
    #running loop for tweets
    
    all_tweets = []

    some_tweets = []

    for i in range(4):

        for handle in newtwitter:

            rule = gen_rule_payload(handle + "-is:retweet",
                                    from_date = start,
                                    to_date = end,
                                    results_per_call = 500)

            tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

            [print(tweet.all_text) for tweet in tweets2[0:10]];
        
            some_tweets.extend(tweets2)
        
            time.sleep(1)
        
        all_tweets.extend(some_tweets)
        
        
        
    #creating df    
    
    
    
    # We create a pandas dataframe as follows:
    data_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets], columns=['Tweets'])
    
    #adding more columns
    data_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets])
    data_tweets['ID']   = np.array([tweet.id for tweet in all_tweets])
    data_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets])
    data_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets])
    data_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets])
    data_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets])
    data_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets])
    data_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets])
    data_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets])
    data_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets])
    data_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets])
    data_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets])
    data_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets])
    data_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets])
    data_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets])
    
    return data_tweets

In [109]:
teams = ["Kansas City Chiefs", "Tampa Bay Buccaneers",
                                "Baltimore Ravens", "Philadelphia Eagles",
                                "Carolina Panthers"]
data_t1_1 = get_data(teams)

In [104]:
get_tweets(data_t1_1)

['Cam Newton OR @CameronNewton', 'Devin Funchess OR @D_FUNCH', 'Christian McCaffrey OR @run__cmc', 'Jameis Winston OR @Jaboowins', 'Mike Evans OR @MikeEvans13_', 'Adam Humphries OR @adamhumphries13', 'Carson Wentz OR @cj_wentz', 'Zach Ertz OR @ZERTZ_86', 'Alshon Jeffrey OR @TheWorldof_AJ', 'Alex Smith', 'Travis Kelce OR @tkelce', 'Tyreek Hill OR @cheetah', 'Joe Flacco OR @JoeFlacco', 'Nick Boyle OR @nickboyle86', 'Mike Wallace OR @Wallace17_daKid']
2017-09-17 18:00
2017-09-17 22:00


,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Christian McCaffrey OR @run__cmc'
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Jameis Winston OR @Jaboowins'
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Mike Evans OR @MikeEvans13_'
11,Adam Humphries,@adamhumphries13',Tampa Bay Buccaneers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Adam Humphries OR @adamhumphries13'
12,Carson Wentz,@cj_wentz',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Carson Wentz OR @cj_wentz'
13,Zach Ertz,@ZERTZ_86',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Zach Ertz OR @ZERTZ_86'
14,Alshon Jeffrey,@TheWorldof_AJ',Philadelphia Eagles,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Alshon Jeffrey OR @TheWorldof_AJ'
15,Alex Smith,NaN,Kansas City Chiefs,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 21:25',2017-09-25 01:25',2017-10-03 01:30',2017-10-03 05:30',2017-10-09 01:30',2017-10-09 05:30',2017-10-15 21:25',2017-10-16 01:25',Alex Smith


In [ ]:
####T1 - 1

In [ ]:
#picking out the players that I want

In [61]:
data_t1_1 = data[data.Team.isin(["Kansas City Chiefs", "Tampa Bay Buccaneers",
                                "Baltimore Ravens", "Philadelphia Eagles",
                                "Carolina Panthers"])]

In [62]:
data_t1_1.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Christian McCaffrey OR @run__cmc'
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Jameis Winston OR @Jaboowins'
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Mike Evans OR @MikeEvans13_'


In [28]:
data_t1_1

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@CameronNewton' OR Cam Newton
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@D_FUNCH' OR Devin Funchess
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@run__cmc' OR Christian McCaffrey
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',@Jaboowins' OR Jameis Winston
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',@MikeEvans13_' OR Mike Evans
11,Adam Humphries,@adamhumphries13',Tampa Bay Buccaneers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',@adamhumphries13' OR Adam Humphries
12,Carson Wentz,@cj_wentz',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@cj_wentz' OR Carson Wentz
13,Zach Ertz,@ZERTZ_86',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@ZERTZ_86' OR Zach Ertz
14,Alshon Jeffrey,@TheWorldof_AJ',Philadelphia Eagles,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',@TheWorldof_AJ' OR Alshon Jeffrey
15,Alex Smith,NaN,Kansas City Chiefs,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 21:25',2017-09-25 01:25',2017-10-03 01:30',2017-10-03 05:30',2017-10-09 01:30',2017-10-09 05:30',2017-10-15 21:25',2017-10-16 01:25',NaN


In [ ]:
#getting rid of the quotations

In [63]:
twitter_t1_1 = data_t1_1.full_twitter.tolist()

newtwitter_t1_1 = []

for i in range(len(twitter_t1_1)):
    a = twitter_t1_1[i].replace("'", "")
    
    newtwitter_t1_1.append(a)
    
print(newtwitter_t1_1)

['Cam Newton OR @CameronNewton', 'Devin Funchess OR @D_FUNCH', 'Christian McCaffrey OR @run__cmc', 'Jameis Winston OR @Jaboowins', 'Mike Evans OR @MikeEvans13_', 'Adam Humphries OR @adamhumphries13', 'Carson Wentz OR @cj_wentz', 'Zach Ertz OR @ZERTZ_86', 'Alshon Jeffrey OR @TheWorldof_AJ', 'Alex Smith', 'Travis Kelce OR @tkelce', 'Tyreek Hill OR @cheetah', 'Joe Flacco OR @JoeFlacco', 'Nick Boyle OR @nickboyle86', 'Mike Wallace OR @Wallace17_daKid']


In [ ]:
#checking to see if it worked

In [64]:
len(newtwitter_t1_1)

15

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [71]:
start_t1_1 = data_t1_1.T1_start.tolist()

start_t1_1 = start_t1_1[1].replace("'", "")
    
print(start_t1_1)

2017-09-17 18:00


In [ ]:
#end

In [73]:
end_t1_1 = data_t1_1.T1_end.tolist()

end_t1_1 = end_t1_1[1].replace("'", "")
    
print(end_t1_1)
type(end_t1_1)

2017-09-17 22:00


str

In [ ]:
#running loop T1 - 1

In [74]:
all_tweets_t1_1 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_1:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_1,
                                to_date = end_t1_1,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_1.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#should be 10*15*10 = 1500

In [76]:
len(all_tweets_t1_1)

30

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_1_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_1], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_1_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [79]:
data_t1_1_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_1])
data_t1_1_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_1])
data_t1_1_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_1])
data_t1_1_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_1])
data_t1_1_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_1])
data_t1_1_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_1])
data_t1_1_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_1])
data_t1_1_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_1])
data_t1_1_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_1])
data_t1_1_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_1])
data_t1_1_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_1])
data_t1_1_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_1])
data_t1_1_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_1])
data_t1_1_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_1])
data_t1_1_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_1])

In [80]:
display(data_t1_1_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text,retweeted_tweet,user_mentions,profile_location,in_reply_to_screen_name,created_at_string,tweet_type,all_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",,{'created_at': 'Sun Sep 17 21:37:56 +0000 2017...,"[{'screen_name': 'PFF_NateJahnke', 'name': 'Na...",None,None,2017-58-17T21:58:47.000Z,retweet,"Ravens final WR snap count: Mike Wallace 48, J..."
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",,{'created_at': 'Sun Sep 17 21:37:56 +0000 2017...,"[{'screen_name': 'PFF_NateJahnke', 'name': 'Na...",None,None,2017-56-17T21:56:49.000Z,retweet,"Ravens final WR snap count: Mike Wallace 48, J..."
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",,{'created_at': 'Sun Sep 17 21:37:56 +0000 2017...,"[{'screen_name': 'PFF_NateJahnke', 'name': 'Na...",None,None,2017-38-17T21:38:32.000Z,retweet,"Ravens final WR snap count: Mike Wallace 48, J..."
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J...",None,[],None,None,2017-37-17T21:37:56.000Z,tweet,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc...",None,"[{'screen_name': 'OmarKelly', 'name': 'Omar Ke...",None,OmarKelly,2017-33-17T21:33:16.000Z,tweet,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...,None,[],None,None,2017-15-17T21:15:28.000Z,tweet,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace,None,"[{'screen_name': 'HewalkstheSky', 'name': '.',...",None,HewalkstheSky,2017-00-17T21:00:40.000Z,tweet,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...,None,"[{'screen_name': 'MatthewBerryTMR', 'name': 'M...",None,MatthewBerryTMR,2017-47-17T20:47:26.000Z,tweet,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash,None,[],None,None,2017-34-17T20:34:13.000Z,tweet,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...,None,[],None,None,2017-23-17T20:23:15.000Z,tweet,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T1 - 2

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T1 - 3

In [ ]:
#picking out the players that I want

In [92]:
data_t1_3 = data[data.Team.isin(["Seattle Seahawks"])]

In [93]:
data_t1_3.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
0,Russell Wilson,@DangeRussWilson',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Russell Wilson OR @DangeRussWilson'
1,Doug Baldwin,@DougBaldwinJr',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Doug Baldwin OR @DougBaldwinJr'
2,Jimmy Graham,@TheJimmyGraham',Seattle Seahawks,W,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Jimmy Graham OR @TheJimmyGraham'


In [85]:
data_t1_3

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [94]:
twitter_t1_3 = data_t1_3.full_twitter.tolist()

newtwitter_t1_3 = []

for i in range(len(twitter_t1_3)):
    a = twitter_t1_3[i].replace("'", "")
    
    newtwitter_t1_3.append(a)
    
print(newtwitter_t1_3)

['Russell Wilson OR @DangeRussWilson', 'Doug Baldwin OR @DougBaldwinJr', 'Jimmy Graham OR @TheJimmyGraham']


In [ ]:
#checking to see if it worked

In [95]:
len(newtwitter_t1_3)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [96]:
start_t1_3 = data_t1_3.T1_start.tolist()

start_t1_3 = start_t1_3[1].replace("'", "")
    
print(start_t1_3)

2017-09-10 21:25


In [ ]:
#end

In [97]:
end_t1_3 = data_t1_3.T1_end.tolist()

end_t1_3 = end_t1_3[1].replace("'", "")
    
print(end_t1_3)
type(end_t1_3)

2017-09-11 01:25


str

In [ ]:
#running loop T1 - 3

In [74]:
all_tweets_t1_3 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_3:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_3,
                                to_date = end_t1_3,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_3.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_3)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_3_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_3], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_3_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_3_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_3])
data_t1_3_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_3])
data_t1_3_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_3])
data_t1_3_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_3])
data_t1_3_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_3])
data_t1_3_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_3])
data_t1_3_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_3])
data_t1_3_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_3])
data_t1_3_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_3])
data_t1_3_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_3])
data_t1_3_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_3])
data_t1_3_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_3])
data_t1_3_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_3])
data_t1_3_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_3])
data_t1_3_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_3])

In [78]:
display(data_t1_3_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T1 - 4

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T2-1

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T2-2

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T2-3

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T2-4

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
####T2-5

In [ ]:
#picking out the players that I want

In [83]:
data_t1_2 = data[data.Team.isin(["Dallas Cowboys"])]

In [84]:
data_t1_2.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [85]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting rid of the quotations

In [86]:
twitter_t1_2 = data_t1_2.full_twitter.tolist()

newtwitter_t1_2 = []

for i in range(len(twitter_t1_2)):
    a = twitter_t1_2[i].replace("'", "")
    
    newtwitter_t1_2.append(a)
    
print(newtwitter_t1_2)

['Dak Prescott OR @dak', 'Dez Bryant OR @DezBryant', 'Jason Witten OR @JasonWitten']


In [ ]:
#checking to see if it worked

In [88]:
len(newtwitter_t1_2)

3

In [ ]:
#getting final start and end dates

In [ ]:
#start

In [89]:
start_t1_2 = data_t1_2.T1_start.tolist()

start_t1_2 = start_t1_2[1].replace("'", "")
    
print(start_t1_2)

2017-09-11 01:30


In [ ]:
#end

In [90]:
end_t1_2 = data_t1_2.T1_end.tolist()

end_t1_2 = end_t1_2[1].replace("'", "")
    
print(end_t1_2)
type(end_t1_2)

2017-09-11 05:30


str

In [ ]:
#running loop T1 - 2

In [74]:
all_tweets_t1_2 = []

some_tweets = []

for i in range(4):

    for handle in newtwitter_t1_2:

        rule = gen_rule_payload(handle + "-is:retweet",
                                from_date = start_t1_2,
                                to_date = end_t1_2,
                                results_per_call = 500)

        tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

        [print(tweet.all_text) for tweet in tweets2[0:10]];
        
        some_tweets.extend(tweets2)
        
        time.sleep(1)
        
    all_tweets_t1_2.extend(some_tweets)

Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam is at the podium

Listen Live ➡️https://t.co/dhD5HZOQig https://t.co/PVvJjMdd7v
.@JKCorden worked a shift at @OldNavy and may have changed the retail game forever.

WATCH: https://t.co/NB59bnasAq https://t.co/W4UHewVUCT
Sunday FreakOut: Patriots are back, Carolina's 2-0 and in trouble, Travis Kelce does a dumb thing then a good thing https://t.co/xl6LQcd2i9
Cam Newton throwing these passes like https://t.co/oXsZKuw1Zw
Cam Newton: “I’m disappointed in myself but happy for the whole team.”
@NFL @CameronNewton @Panthers He got some crusty feet and he dresses like a clown.
You need a pair of track pants in your off-duty (and on-duty) wardrobe https://t.co/7EM1x0c3E2
Postgame with @CameronNewton 🎩 #KeepPounding https://t.co/bnBUZqLxcG
Cam Newton (ankle) will return today.
@D_FUNCH  Excellent game DF! Loved seeing you make those tough catches.
Cam Newton threw eight balls at least 16 yards downfield today. Five of them 

Here's Alex Smith on Travis Kelce and his passion on the football field. #ChiefsKingdom https://t.co/W9GBRNwr6n
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Alex Smith has to stop this 9 yards per attempt shit if I'm gonna watch Mahomes this decade
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
@SuperStatsDave That was Alex Smith before Harbaugh as well
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Vinny Curry on failing to bring down Alex Smith: "I had him. He broke loose. If we play again, he won't break loose." #Eagles
Antonio Brown, Martavis Bryant, and JuJu Smith-Schuster all scored touchdowns on their first career touch.

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Tried trading Odell for Mike Evans and got shut down 😪😂
.@MikeEvans13_ wearing a @DwyaneWade jersey as he gears up for #CHIvsTB @MiamiHEAT https://t.co/QcV3fYcGlQ
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last tim

@MatthewBerryTMR  dope insight on nick boyle. You're the goat.
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
Ravens final WR snap count: Mike Wallace 48, Jeremy Maclin 43, Breshad Perriman 42, Michael Campanaro 8, Chris Matthews 7. Out of 68
@OmarKelly That bubble screen worked once, once with Mike Wallace in '13 in Indy.
Sto-ology: Aldous Huxley interviewed by Mike Wallace : 1958 (... https://t.co/5tAr9Dlg3J
@HewalkstheSky Andy Dalton and mike wallace
@MatthewBerryTMR time to push the panic button on mike Wallace and isiah crowell?
Mike Wallace is trash
Buoni progressi di #Flacco (non che ci volesse molto), ma pochi i target a disposizion

.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's first TD in an #Eagles' uniform. #FlyEaglesFly https://t.co/nXwKe0rU52
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
@ricohern2905 @TheWorldof_AJ Exactly, I'm very excited about this team!!!!!
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC https://t.co/osnB8CcFTD
Retweet #LFGTouchdownSweepstakes for your chance to win sweet gear thanks to that @TheWorldof_AJ touchdown! https://t.co/vbeambejyj
.@TheWorldof_AJ's FIRST TD with the @Eagles! #FlyEaglesFly #PHIvsKC htt

The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
The @TBBuccaneers @MikeEvans13_  and @Jaboowins are BALLIN' today. 🏀 #NFLCelebrations https://t.co/MMQUL3Zofq
Jameis Winston to Mike Evans for the 13 yard TD! 

Not the last time we'll hear that. https://t.co/saMXQ9vGBl
Beautiful ball from Jameis Winston to O.J. Howard. https://t.co/GhexmCacV1
The @TBBuccaneers @MikeEvans13_  and @Jaboowi

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'Carson Wentz OR @cj_wentz', 'maxResults': 10, 'toDate': '201709172200', 'fromDate': '201709171800'}


HTTPError: 

In [ ]:
#how many tweets

In [91]:
len(all_tweets_t1_2)

NameError: name 'all_tweets_t1_2' is not defined

In [ ]:
#creating df

In [75]:
# We create a pandas dataframe as follows:
data_t1_2_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets_t1_2], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data_t1_2_tweets.head(10))

,Tweets
0,RT @PFF_NateJahnke: Ravens final WR snap count...
1,RT @PFF_NateJahnke: Ravens final WR snap count...
2,RT @PFF_NateJahnke: Ravens final WR snap count...
3,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...


In [ ]:
#adding more columns

In [77]:
data_t1_2_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets_t1_2])
data_t1_2_tweets['ID']   = np.array([tweet.id for tweet in all_tweets_t1_2])
data_t1_2_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets_t1_2])
data_t1_2_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets_t1_2])
data_t1_2_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets_t1_2])
data_t1_2_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets_t1_2])
data_t1_2_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets_t1_2])
data_t1_2_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets_t1_2])
data_t1_2_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets_t1_2])
data_t1_2_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets_t1_2])
data_t1_2_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets_t1_2])
data_t1_2_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets_t1_2])

In [78]:
display(data_t1_2_tweets.head(10))

,Tweets,len,ID,Date,Likes,RTs,Quoted,Q_or_RT,User_ent_text
0,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909537142119989249,2017-09-17 21:58:47,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
1,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909536644205830145,2017-09-17 21:56:49,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
2,RT @PFF_NateJahnke: Ravens final WR snap count...,140,909532045394743298,2017-09-17 21:38:32,0,0,None,"Ravens final WR snap count: Mike Wallace 48, J...",
3,"Ravens final WR snap count: Mike Wallace 48, J...",132,909531892726235136,2017-09-17 21:37:56,7,3,None,,"Ravens final WR snap count: Mike Wallace 48, J..."
4,"@OmarKelly That bubble screen worked once, onc...",81,909530720082759681,2017-09-17 21:33:16,0,0,None,,"@OmarKelly That bubble screen worked once, onc..."
5,Sto-ology: Aldous Huxley interviewed by Mike W...,88,909526241438511106,2017-09-17 21:15:28,0,0,None,,Sto-ology: Aldous Huxley interviewed by Mike W...
6,@HewalkstheSky Andy Dalton and mike wallace,43,909522516200144896,2017-09-17 21:00:40,0,0,None,,@HewalkstheSky Andy Dalton and mike wallace
7,@MatthewBerryTMR time to push the panic button...,81,909519185549697030,2017-09-17 20:47:26,0,0,None,,@MatthewBerryTMR time to push the panic button...
8,Mike Wallace is trash,21,909515859781083136,2017-09-17 20:34:13,0,0,None,,Mike Wallace is trash
9,Buoni progressi di #Flacco (non che ci volesse...,137,909513097798438912,2017-09-17 20:23:15,0,0,None,,Buoni progressi di #Flacco (non che ci volesse...
